# 천이과정 학습 코드
본 Jupyter notebook은 UAM 스케일 기체의 천이과정을 학습시키는 코드입니다.

### 라이브러리 import

In [1]:
import gym
import torch as th
import time
import Transition_Training_8_discrete_simple as TT8d
import importlib
from stable_baselines3 import SAC
from datetime import datetime

**Transition_Training_8_discrete_yuc** 파일을 수정하고 다시 불러올 경우, 아래의 셀을 실행시킵니다.

In [11]:
importlib.reload(TT8d)

<module 'Transition_Training_8_discrete_simple' from 'f:\\KADA UAM RL\\code\\Transition_Training_8_discrete_simple.py'>

### 파일 이름 생성
아래 함수를 실행시킨 시점을 기준으로 파일 이름을 반환합니다.

In [3]:
def get_filename():
    year = str(datetime.today().year)
    month = str(datetime.today().month).zfill(2)
    day = str(datetime.today().day).zfill(2)
    hour = str(datetime.today().hour).zfill(2)
    minute = str(datetime.today().minute).zfill(2)
    second = str(datetime.today().second).zfill(2)

    filename = f'train_result_{year}-{month}-{day}_{hour}{minute}{second}'
    return filename

### 모델 생성
처음부터 모델을 학습 시킬 경우, 아래의 셀을 실행시킵니다.  
이전에 학습시킨 모델을 불러와 전이학습을 시키려면 [모델 불러오기](#모델-불러오기) 셀을 확인하세요

In [ ]:
# Parallel environments
env = TT8d.TiltrotorTransitionTraining()
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[256,256,256], qf=[128,128]))
model = SAC("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=1, device="cuda")

### 모델 불러오기
기존에 학습된 모델이 있다면 불러와서 전이학습을 시킬 수도 있습니다.  
아래의 셀을 실행시켜 모델을 불러옵니다.

In [9]:
env = TT8d.TiltrotorTransitionTraining()
model = SAC.load(r'./model/train_result_2023-07-25_225825_last.zip', env = env, device='cuda')

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\cck18\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


### 학습시키기
아래의 코드를 동작시켜 모델을 학습시킵니다.

In [10]:
importlib.reload(TT8d)

maximum_eps = 1000
maximum_timestep = 30000

best_reward = 0

result_filename = get_filename()
with open(f'./results/{result_filename}.txt', 'w') as file:
    for eps in range(maximum_eps):
        obs = env.reset()
        model.learn(total_timesteps=maximum_timestep, log_interval=100)
        for k in range(maximum_timestep):
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            env.render()
            if done:
                log = f"episode: {eps}\nepisode was finished at timestep: {k}\nreward: {reward}\n========================================\n" + \
                f"reward details\n" + \
                f"reward_1: {info['reward_detail'][0]}\n" + \
                f"reward_2: {info['reward_detail'][1]}\n" + \
                f"reward_3: {info['reward_detail'][2]}\n" + \
                f"reward_4: {info['reward_detail'][3]}\n" + \
                f"reward_5: {info['reward_detail'][4]}\n" + \
                f"reward_6: {info['reward_detail'][5]}\n" + \
                "========================================\n"
                print(log)
                file.write(f'{log}\n')

                break
        # 이번 학습 결과가 기존 학습 결과보다 좋다면 저장
        if reward > best_reward:
            best_reward = reward
            model.save(f'./model/{result_filename}')

model.save(f'./model/{result_filename}_last')

ValueError: too many values to unpack (expected 4)

### 모델 시연
학습이 마무리된 후, 아래의 셀을 실행시켜 모델 학습 결과를 확인합니다.

In [ ]:
obs = env.reset()

for k in range(3000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #print(action)
    env.render()
    if done:
      obs = env.reset()

print(model.policy)

In [ ]:
env = TT8d.TiltrotorTransitionTraining()
model = SAC.load(r'./model/train_result_2023-07-13_224219.zip')

In [ ]:
time.sleep(5)
obs = env.reset()

for k in range(1000):
    action, _states = model.predict(obs, deterministic=True)

    obs, reward, done, info = env.step(action)
    #print(action)
    env.render()
    if done:
      obs = env.reset()

print(model.policy)